In [ ]:
%pip install graphdatascience

## Setup GDS graph

In [2]:
import os

from graphdatascience import GraphDataScience

# Get Neo4j DB URI, credentials and name from environment if applicable
NEO4J_URI = os.environ.get("NEO4J_URI", "bolt://localhost:7687")
NEO4J_AUTH = ("neo4j", None)
NEO4J_DB = os.environ.get("NEO4J_DB", "neo4j")
if os.environ.get("NEO4J_USER") and os.environ.get("NEO4J_PASSWORD"):
    NEO4J_AUTH = (
        os.environ.get("NEO4J_USER"),
        os.environ.get("NEO4J_PASSWORD"),
    )
gds = GraphDataScience(NEO4J_URI, auth=NEO4J_AUTH, database=NEO4J_DB)

In [3]:
for g in gds.graph.list()["graphName"]:
    gds.graph.drop(g)

In [ ]:
G_imdb = gds.graph.load_cora()
# G_imdb = gds.graph.load_imdb()

In [ ]:
gds.pageRank.mutate(G_imdb, mutateProperty="pagerank")

In [ ]:
# Compute communities

gds.louvain.mutate(G_imdb, mutateProperty="componentId")

In [7]:
G_sample, _ = gds.graph.sample.rwr("sampled_2", G_imdb)

In [8]:
## Prepare input for nevada

In [ ]:
actual_node_properties = list(G_imdb.node_properties().to_dict().values())[0]
actual_node_properties

In [ ]:
node_df = gds.graph.nodeProperties.stream(
    G_sample, actual_node_properties, listNodeLabels=True, separate_property_columns=True
)

node_df

In [ ]:
# Rescale for size
scores = node_df["pagerank"]

node_df["pagerank"] = scores / scores.min()
node_df["pagerank"]

In [ ]:
# map community to color
import colorsys
import random


@staticmethod
def _random_themed_color(theme) -> str:
    label = 0.7 if theme == "dark" else 0.4
    return "#%02X%02X%02X" % tuple(map(lambda x: int(x * 255), colorsys.hls_to_rgb(random.random(), label, 1.0)))


color_dict = {component: _random_themed_color("dark") for _, component in node_df["componentId"].items()}
color_dict

In [13]:
def map_node(n):
    caption = ""
    for label in n["nodeLabels"]:
        caption += label
    # TODO: scale pageRank values
    return {"id": str(n["nodeId"]), "caption": caption, "size": n["pagerank"], "color": color_dict[n["componentId"]]}


nodes = [map_node(node) for _, node in node_df.iterrows()]

In [14]:
rel_df = gds.graph.relationships.stream(G_sample)

In [15]:
def map_relationship(idx, r):
    return {
        "id": str(idx),
        "from": str(r["sourceNodeId"]),
        "to": str(r["targetNodeId"]),
        "caption": str(r["relationshipType"]),
    }


relationships = [map_relationship(idx, rel) for idx, rel in rel_df.iterrows()]

In [16]:
## Visualize with Nevada

In [ ]:
from neo4j_viz.nvl import NVL

my_nvl = NVL()

my_nvl

In [ ]:
G_sample

In [ ]:
my_nvl.render(nodes=nodes, relationships=relationships)